# 5.i. BDR-specific querying

Int his module we will test SPARQL queries on the Biodiversity Data Repository (BDR) system.

Before training with this module, a revision of Modules 3 & 4 should be done first.

### Data Access

This module requires that queries be executed on a SPARQL Endpoint with access to BDR data. While public access to some demo BDR data is available, there may not be public access to the cull BDR content.

> **NOTE**: A BDR full content endpoint will be provisioned for this work, but it will not be recorded here for general, public, access.

## Outline

1. Test access as usual
2. Approaching a DB
    * generic approaches to discovering what's in any large RDF DB
2. Approaching the BDR via ABIS / BDR PR
3. BDR Queries
    * partly prepared by Ashley
4. Desktop GIS SPARQL Use

## 5.i.0 Test access as usual

In [ ]:
# import some Python packages
from kurra.sparql import query
from IPython.display import display, Markdown
from kurra.utils import render_sparql_result

# define our pretty-print function
def table_print(r):
    display(Markdown(render_sparql_result(r)))

In [ ]:
bdr_sparql_endpoint = "XXXX"

# make a simple SPARQL query
q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT *
WHERE {
  ?o a owl:Ontology .

  OPTIONAL {
    ?o rdf:label|skos:prefLabel ?label
  }
}
"""

r = query(bdr_sparql_endpoint, q)
table_print(r)

In [ ]:
# Force a timeout, maybe...?
q = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT *
WHERE {
  ?o a skos:Concept .

  OPTIONAL {
    ?o skos:prefLabel ?label
  }
}
"""

q2 = """
PREFIX schema: <https://schema.org/>

SELECT *
WHERE {
  ?o a schema:Dataset

  OPTIONAL {
    ?o schema:name ?label
  }
}
ORDER BY DESC(?label)
"""

r = query(bdr_sparql_endpoint, q)
table_print(r)  # 115,868 Concepts, 10k+ Datasets

In [ ]:
# Respect the timeout
q = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT *
WHERE {
  ?o a skos:Concept .

  OPTIONAL {
    ?o skos:prefLabel ?label
  }
}
LIMIT 10
"""

r = query(bdr_sparql_endpoint, q)
table_print(r)

### 5.i.1 Approaching a DB

#### Look for definitional items

* Search for any `owl:Ontology` instances
* Search for any `skos:ConceptScheme` instances

#### View the BD structure

* Search for Named Graphs

```sparql
SELECT DISTINCT ?g
WHERE {
  GRAPH ?g {
   	?s ?p ?o
  }
}
LIMIT 100
```

Then try to `DESCRIBE` a graph.

### 5.i.2 Approaching the BDR via ABIS / BDR PR

#### Read the documentation

In this case, start at the [BDR Resources Page](https://resources.bdr.gov.au/)

* [ABIS](https://linked.data.gov.au/def/abis)
* [BDR Profile of ABIS](https://linked.data.gov.au/def/bdr-pr)
* [BDR Catalogues](https://resources.bdr.gov.au/catalogues)

#### Ask someone

The BDR Team!

### 5.i.3 BDR Queries

#### Count Occurrences

```sparql
SELECT (COUNT(?o) AS ?count)
WHERE {
	?o a <http://rs.tdwg.org/dwc/terms/Occurrence>
}
```

#### Count occurrences in a Dataset

![](Dataset_Records.svg)

```sparql
PREFIX abis: <https://linked.data.gov.au/def/abis/>
PREFIX schema: <https://schema.org/>
SELECT ?dataset (COUNT(?rec) as ?record_count) {
    BIND (<https://linked.data.gov.au/dataset/bdr/63c890eb-cd66-4cb5-8518-45bd6dc1409c> as ?ds)
    ?dataset a schema:Dataset .
    ?rec a abis:BiodiversityRecord ;
      schema:isPartOf ?dataset .
} GROUP BY ?dataset
```

#### Find `Dataset` instances, and some metadata

```sparql
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>
SELECT ?dataset ?ds_name ?provider ?provider_name WHERE {
    ?dataset a schema:Dataset .
    ?dataset schema:name ?ds_name ;
      prov:qualifiedAttribution [
        prov:hadRole <https://linked.data.gov.au/def/data-roles/resourceProvider> ;
        prov:agent ?provider
      ] .
    ?provider schema:name ?provider_name .
} LIMIT 1000 # Limit because there are over 10,000 ds
```

#### Count the records

In every Dataset provided by QLD WildNet

(Find the qld wildnet URI by looking in the BDR Orgs Catalogue!)

```sparql
PREFIX abis: <https://linked.data.gov.au/def/abis/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>
SELECT ?dataset (COUNT(?rec) as ?record_count) WHERE {
    ?dataset a schema:Dataset ;
      prov:qualifiedAttribution [
        prov:hadRole <https://linked.data.gov.au/def/data-roles/resourceProvider> ;
        prov:agent <https://linked.data.gov.au/org/qld-wildnet>
      ] .
    ?rec a abis:BiodiversityRecord .
    ?rec schema:isPartOf ?dataset .
} GROUP BY ?dataset
LIMIT 1000 # Must limit to 1000 because counting in graphs is SLOW!
```

#### Coarse Geospatial search.

Find which Datasets bounding boxes intersect the bounding box of K'gari

![](kgari_box.png)

```sparql
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?dataset WHERE {
    BIND("""POLYGON ((152.935181 -25.787527, 153.382874 -25.787527, 153.382874 -24.684457, 152.935181 -24.684457, 152.935181 -25.787527))"""^^geo:wktLiteral as ?Kgari_bbox)
    ?dataset a schema:Dataset .
    ?dataset geo:hasBoundingBox ?geo .
    ?geo geo:asWKT ?bounds
    FILTER geof:sfIntersects(?bounds, ?Kgari_bbox)
}
LIMIT 1000
```

#### Finer Geospatial search

Find which Dataasets (using their convex hull) contain occurrences on K'gari polygon

![](kgari_hull.png)

```sparql
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?dataset WHERE {
    BIND("""POLYGON ((153.231812 -24.666986, 153.045044 -24.791722, 153.198853 -24.96614, 152.946167 -25.207426, 153.014832 -25.34899, 152.896729 -25.532528, 152.946167 -25.703413, 152.987366 -25.79, 153.099976 -25.819672, 153.388367 -25.000994, 153.316956 -24.66449, 153.231812 -24.666986))"""^^geo:wktLiteral as ?Kgari_polygon)
    ?dataset a schema:Dataset .
    ?dataset geo:hasGeometry ?geo .
    ?geo geo:asWKT ?bounds
    FILTER geof:sfIntersects(?bounds, ?Kgari_polygon)
}
LIMIT 100
```

### National Species List Use

NSL relationships in the BDR (This is using the official published RDF version of the NSL)

<div>
    <img src="module-5i-files/NSL_Toplevel.svg" width="75%"/>
</div>


#### Find which datasets containing Species

_Velleia perfoliata R.Br._

![](NameUsageSubject.svg)

```sparql
PREFIX schema: <https://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX nsl: <https://linked.data.gov.au/def/nsl/>
SELECT ?dataset
WHERE {
    {
        SELECT ?nsl_usage
        WHERE {
            GRAPH <https://biodiversity.org.au/nsl/2025-06-16> {
                ?nsl_name
                    a nsl:TaxonName ;
                    rdfs:label "Velleia perfoliata R.Br." ;
                .

                ?nsl_usage nsl:usageOf ?nsl_name .
            }
        }
    }

    ?dataset a schema:Dataset .

    ?nsl_usage
        schema:subjectOf ?dataset ;
    .
}
LIMIT 100
```


#### Find location and time of all instances of Species

_Velleia perfoliata R.Br._,  within one given Dataset:

```sparql
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX schema: <https://schema.org/>
PREFIX abis: <https://linked.data.gov.au/def/abis/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX nsl: <https://linked.data.gov.au/def/nsl/>
SELECT ?record ?id ?occurrence ?location ?datetime
WHERE {
    {
        SELECT ?nsl_usage
        WHERE {
            GRAPH <https://biodiversity.org.au/nsl/2025-06-16> {
                ?nsl_name
                    a nsl:TaxonName ;
                    rdfs:label "Velleia perfoliata R.Br." ;
                .

                ?nsl_usage nsl:usageOf ?nsl_name .
            }
        }
    }

    BIND (<https://linked.data.gov.au/dataset/bdr/c1130d9b-2655-4454-bab1-9c3cebe381dc> as ?dataset)

    ?record
        a abis:BiodiversityRecord ;
        schema:identifier ?id ;
        schema:isPartOf ?dataset ;
        schema:about ?occurrence ;
    .
    ?nsl_usage schema:subjectOf ?occurrence .
    ?occurrence
        schema:spatial ?spatial ;
        schema:temporal ?temporal ;
    .

    ?spatial geo:asWKT ?location .

    ?temporal time:inXSDDate|time:inXSDDateTime|time:inXSDDateTimeStamp|time:inXSDgYear|time:inXSDgYearMonth ?datetime .
}
LIMIT 100
```

#### Show all observations on a given occurrence

(this shows the values of observable properties at a given time)

![](ObservationsObservationCollections.svg)

```sparql
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX schema: <https://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT ?record ?observation ?property ?property_label ?procedure ?procedure_label ?result ?time_val ?time_wkt ?comment
WHERE {
    BIND (<https://linked.data.gov.au/dataset/bdr/c1130d9b-2655-4454-bab1-9c3cebe381dc/biodiversityRecord/7768> as ?record)

    ?record schema:about ?occurrence .

    ?observation
        sosa:hasFeatureOfInterest ?occurrence ;
        sosa:observedProperty ?property ;
        sosa:usedProcedure ?procedure ;
        sosa:hasSimpleResult ?result ;
    .

    ?property skos:prefLabel ?property_label .

    ?procedure skos:prefLabel ?procedure_label .

    OPTIONAL {
        ?observation schema:temporal|sosa:phenomenonTime ?time_node .
        ?time_node time:inXSDDate|time:inXSDDateTime|time:inXSDDateTimeStamp|time:inXSDgYear|time:inXSDgYearMonth ?time_val .
    }
    OPTIONAL {
        ?observation schema:spatial ?spatial_node .
        ?spatial geo:asWKT ?spatial_wkt .
    }
    OPTIONAL {
        ?observation schema:comment ?comment .
    }
}
LIMIT 10
```

#### Get all the Attributes

Using `tern:hasAttribute`, for an Occurrence, or attributes of the Occurrence the record is about.

```sparql
PREFIX tern: <https://w3id.org/tern/ontologies/tern/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX schema: <https://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?record ?attr ?attr_label ?val
WHERE {
    BIND (<https://linked.data.gov.au/dataset/bdr/c1130d9b-2655-4454-bab1-9c3cebe381dc/biodiversityRecord/7768> as ?record)

    ?record schema:about ?occurrence .
    {
        ?occurrence tern:hasAttribute ?attr_node .
    }
    UNION
    {
        ?record tern:hasAttribute ?attr_node .
    }
    UNION
    {
        ?attr_collection
            tern:hasAttribute ?attr_node ;
            schema:hasPart ?record ;
        .
    }
    UNION
    {
        ?attr_collection
            tern:hasAttribute ?attr_node ;
            schema:hasPart ?occurrence ;
        .
    }

    ?attr_node
        tern:attribute ?attr ;
        tern:hasSimpleValue ?val ;
    .

    OPTIONAL {
        ?attr skos:prefLabel ?attr_label .
    }
}
LIMIT 10
```

## 5.i.4 Desktop GIS SPARQL Use

Using QGIS & SPARQLing Unicorn

[QGIS](https://qgis.org/) is a free and open-source desktop GIS environment, similar to ArcGIS Pro.

The [SPARQLing Unicorn Plugin](https://plugins.qgis.org/plugins/sparqlunicorn/) is a QGIS Plugin that "GeoJSON layer from SPARQL endpoint queries".

_we will do a manual demo_